## Visualisation des données

In [3]:
# Importation des librairies nécessaires
import pandas as pd
from collections import Counter


import plotly.express as px

import matplotlib.pyplot as plt
import seaborn as sns

from filmsapisdk import MovieClient, MovieConfig

import time

import json

from pathlib import Path


In [4]:
# Dossiers
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)


In [6]:
# Connexion à l'API via l'ADK

config = MovieConfig(movie_base_url="https://datatech.onrender.com")
client = MovieClient(config=config)

client.health_check()

MOVIE_API_BASE_URL in MovieConfig init: https://datatech.onrender.com


{'Message': 'API MovieLens est opérationnelle!'}

In [7]:
# Récuperation des statistiques analytiques de l'API
analytics = client.get_analytics()
analytics

AnalyticsResponse(movie_count=9742, rating_count=100836, tag_count=3683, link_count=9742)

## Top 10 des genres par nombre de films

In [10]:
# Systeme sans mise en cache
"""# Initialisation du compteur de genres


genre_counter = Counter() 

# parametres pour le batching

limit = 200

skip = 0

while True:

    # Récupération des films par lots
    movies_batch = client.list_movies(limit=limit, skip=skip, output_format='dict')

    if not movies_batch:
        break


    # On extrait les genres de chaque film et on met à jour le compteur
    for movie in movies_batch:
        genres = movie.get("genres", "")
        genre_list = genres.split('|') if genres else []
        genre_counter.update(genre_list)

    skip += limit
    time.sleep(0.5)  # Petite pause pour éviter de surcharger l'API

# Conversion du compteur en DataFrame pour une analyse plus facile
genre_counts_df = pd.DataFrame(genre_counter.items(), columns=['genre', 'count'])   
genre_counts_df = genre_counts_df.sort_values(by='count', ascending=False).head(10)
genre_counts_df



fig = px.bar(
    genre_counts_df,
    x='count',
    y='genre',
    orientation='h',
    title='Distribution des genres de films',
    labels={'count': 'Nombre de films', 'genre': 'Genre'},
    color='count',
    color_continuous_scale='Viridis'
)

fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    height=500
)

fig.show()"""

'# Initialisation du compteur de genres\n\n\ngenre_counter = Counter() \n\n# parametres pour le batching\n\nlimit = 200\n\nskip = 0\n\nwhile True:\n\n    # Récupération des films par lots\n    movies_batch = client.list_movies(limit=limit, skip=skip, output_format=\'dict\')\n\n    if not movies_batch:\n        break\n\n\n    # On extrait les genres de chaque film et on met à jour le compteur\n    for movie in movies_batch:\n        genres = movie.get("genres", "")\n        genre_list = genres.split(\'|\') if genres else []\n        genre_counter.update(genre_list)\n\n    skip += limit\n    time.sleep(0.5)  # Petite pause pour éviter de surcharger l\'API\n\n# Conversion du compteur en DataFrame pour une analyse plus facile\ngenre_counts_df = pd.DataFrame(genre_counter.items(), columns=[\'genre\', \'count\'])   \ngenre_counts_df = genre_counts_df.sort_values(by=\'count\', ascending=False).head(10)\ngenre_counts_df\n\n\n\nfig = px.bar(\n    genre_counts_df,\n    x=\'count\',\n    y=\'genr

In [8]:

# Systeme de mise en cache(evite de faire les meme requetes a l'API si les donnees n'ont pas changé, de consommer les ressources inutilement


api_movie_count = analytics.movie_count
print(f"Nombre total de films dans l'API: {api_movie_count}")

genre_data_file = output_dir / "genre_counts.parquet"
meta_file = output_dir / "genre_counts_meta.json"

# Lecture du fichier meta s'il existe
if meta_file.exists():
    with open(meta_file, 'r') as f:
        meta = json.load(f)
    cached_movie_count = meta.get("movie_count", 0)
else:
    meta = {}

    cached_movie_count = 0

# Décision : utiliser le cache ou recalculer
if genre_data_file.exists() and cached_movie_count == api_movie_count:
    print("Chargegement des données mises en cache.")
    genre_counts_df = pd.read_parquet(genre_data_file)
else:
    print("Mise à jour des données depuis l'API...")
    
    # Initialisation du compteur de genres
    genre_counter = Counter()

    # Parametres pour le batching

    limit = 200
    skip = 0
    while True:
        movies_batch = client.list_movies(limit=limit, skip=skip, output_format='dict')
        if not movies_batch:
            break
        for movie in movies_batch:
            genres = movie.get("genres", "")
            genre_list = genres.split('|') if genres else []
            genre_counter.update(genre_list)
        skip += limit
        time.sleep(0.5)

    # Conversion du compteur en DataFrame
    genre_counts_df = pd.DataFrame(genre_counter.items(), columns=['genre', 'count'])   
    genre_counts_df = genre_counts_df.sort_values(by='count', ascending=False).head(10)

    # Sauvegarde des données mises à jour
    genre_counts_df.to_parquet(genre_data_file,index=False)
    
    with open(meta_file, 'w') as f:
        json.dump({"movie_count": api_movie_count}, f)
    
    # Affichage plotly


fig = px.bar(
        genre_counts_df,
        x='count',
        y='genre',
        orientation='h',
        title='Distribution des genres de films',
        labels={'count': 'Nombre de films', 'genre': 'Genre'},
        color='count',
        color_continuous_scale='Viridis'
    )

fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        height=500
    )

fig.show()
        

Nombre total de films dans l'API: 9742
Chargegement des données mises en cache.


## Nombre total de films par année (basée sur le titre)

In [13]:
import re

# === Dossiers ===

#output_dir = Path("output")
#output_dir.mkdir(exist_ok=True)

yearly_data_file = output_dir / "movies_by_year.parquet"
meta_file = output_dir / "movies_by_year_meta.json"

# === Lecture du nombre total de films via anlytics ===

# analytics = client.get_analytics()

api_movie_count = analytics.movie_count

# === Lecture du cache s'il existe ===

if meta_file.exists():
    with open(meta_file, 'r') as f:
        meta = json.load(f)
    cached_movie_count = meta.get("movie_count", 0)
else:
    cached_movie_count = 0   

# === Utilisation du cache ou recalcul ===

if yearly_data_file.exists() and cached_movie_count == api_movie_count:
    print("chargement des données depuis le cache ...")
    df_yearly = pd.read_parquet(yearly_data_file)
else:
    print("Extraction des données depuis l'API...")
   

    
    # Initialisation 
    year_counter = Counter()
    limit = 200
    skip = 0
    year_pattern = re.compile(r"\((\d{4})\)$")
    while True:
        batch = client.list_movies(limit=limit, skip=skip, output_format='dict')
        if not batch:
            break
        for movie in batch:
            title = movie.get("title", "")
            match = year_pattern.search(title)
            if match:
                year = int(match.group(1))
                year_counter[year] += 1
        skip += limit
        time.sleep(0.5)

    # === Construction du DataFrame ===

    df_yearly = pd.DataFrame(
    sorted(year_counter.items()),
    columns=["year", "movie_count"]
)

  # === Sauvegarde du cache ===

    df_yearly.to_parquet(yearly_data_file, index=False)
    with open(meta_file, "w") as f:
        json.dump({"movie_count": api_movie_count}, f)

 # Affichage plotly


fig = px.bar(
        df_yearly,
        x='year',
        y='movie_count',
        title="Nombre total de films par année (basé sur le titre)",
        labels={"year": "Année", "movie_count": "Nombre de films"},
       
    )

fig.update_layout(
        xaxis_title="Année",
        yaxis_title="Nombre de films",
        height=500
    )

fig.show()

chargement des données depuis le cache ...


## Top 20 des films par nombre d'évaluations

In [ ]:
# === Dossiers ===

#output_dir = Path("output")
#output_dir.mkdir(exist_ok=True)

from collections import defaultdict


top_movie_file = output_dir / "top_movies_by_ratings.parquet"
meta_file = output_dir / "meta_top_movies.json"

# === Récuperation des métriques API ===

# analytics = client.get_analytics()

api_movie_count = analytics.movie_count
api_rating_count = analytics.rating_count

# === Vérification du cache ===

if meta_file.exists():
    with open(meta_file, 'r') as f:
        meta = json.load(f)
    cached_movie_count = meta.get("movie_count", 0)
    cached_rating_count = meta.get("rating_count", 0)
else:
    cached_movie_count = 0
    cached_rating_count = 0

# === Utilisation du cache ou recalcul ===

if (top_movie_file.exists()
    and cached_movie_count == api_movie_count
    and cached_rating_count == api_rating_count
    ):
    print("Chargement des données depuis le cache...")
    top_movies_df = pd.read_parquet(top_movie_file)
   
else:
    print("Récuperation des évaluations depuis l'API...")
    
    # Initialisation des compteurs ===
    movie_rating_count = defaultdict(int)
    movie_rating_sum = defaultdict(float)

    # === Batching des ratings ===
    
    limit = 200
    skip = 0
    
    while True:
        batch = client.list_ratings(limit=limit, skip=skip, output_format='dict')
        if not batch:
            break
        for rating in batch:
            movie_id = rating["movieId"]
            score = rating["rating"]
            movie_rating_count[movie_id] += 1
            movie_rating_sum[movie_id] += score
        skip += limit
        time.sleep(0.5)


    # === Construction du DataFrame ===

    stats = [
        {
            "movieId": movie_id,
            "rating_count": movie_rating_count[movie_id],
            "avg_rating": movie_rating_sum[movie_id] / movie_rating_count[movie_id]
        }
        for movie_id in movie_rating_count
    ]
    stats_df = pd.DataFrame(stats)
    top_movies_df = stats_df.sort_values("rating_count", ascending=False).head(20)

    # === Ajout des titre de films via l'API ===

    movie_titles = {}

    for movie_id in top_movies_df["movieId"]:
        try:
            movie_data = client.get_movie(movie_id)
            movie_titles[movie_id] = movie_data.title
        except Exception as e:
            print(f"Erreur récuperation titre movieId {movie_id} : {e}")
            movie_titles[movie_id] = f"Movie {movie_id}"
        
    top_movies_df["title"] = top_movies_df["movieId"].map(movie_titles)

    # === Sauvegarde dans le cache ===

    top_movies_df.to_parquet(top_movie_file, index=False)
    with open(meta_file, "w") as f:
        json.dump({"movie_count": api_movie_count,
                  "rating_count": api_rating_count
                  },
                f)

 # === Affichage avec plotly ===


fig = px.bar(
        top_movies_df.sort_values("rating_count", ascending=True), # Pour affichage de bas en haut
        x="rating_count",
        y="title",
        color="avg_rating",
        orientation="h",
        title="Top 20 des films par nombre d'évaluations",
        labels={"title": "Titre du film", "rating_count": "Nombre d'évaluations", "avg_rating": "Note moyenne"},
        color_continuous_scale="viridis"
       
)

fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        height=700
    )

fig.show()